In [ ]:
!pip install torch torchvision
!pip install matplotlib numpy pillow
!pip install zennit crp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

from zennit.composites import EpsilonPlusFlat
from zennit.canonizers import SequentialMergeBatchNorm
from crp.attribution import CondAttribution

import torchvision.transforms as T
from PIL import Image
import os
import torch
from torchvision.models.vgg import vgg16_bn
import json

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = vgg16_bn(True).to(device)
model.eval()

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

D:\Programmation\Projet\ExplicationAI\venv\PER\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
D:\Programmation\Projet\ExplicationAI\venv\PER\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
CLASSE_PREDICTED = "classe_predicted"
PROBABILITY = "probability"
FEATURES = "features"

In [ ]:
features_per_layer = {
    0: 64,
    3: 64,
    7: 128,
    10: 128,
    14: 256,
    17: 256,
    20: 256,
    24: 512,
    27: 512,
    30: 512,
    34: 512,
    37: 512,
    40: 512
}

In [ ]:
def compute_feature_importance(model, input_tensor, layer_idx, num_features, pred_class):
    """
    Calcule l'importance de chaque feature d'une couche donnée pour une classe prédite.

    Arguments :
    - model : le modèle VGG16
    - input_tensor : l'image d'entrée sous forme de tenseur
    - layer_idx : l'index de la couche (ex : 40)
    - num_features : le nombre total de features dans cette couche (ex : 512)
    - pred_class : la classe prédite initialement

    Retourne :
    - Un dictionnaire {feature_idx : importance} trié par importance décroissante
    """

    # Obtenir la probabilité originale de la classe prédite
    with torch.no_grad():
        output_original = model(input_tensor)
        probs_original = torch.nn.functional.softmax(output_original, dim=1)
        original_score = probs_original[0, pred_class].item()

    feature_importance = {}

    # Désactiver chaque feature une par une et mesurer l'impact
    for feature_idx in range(num_features):
        def zero_out_feature(module, input, output, feature_idx=feature_idx):
            output[:, feature_idx, :, :] = 0  # Désactiver la feature
            return output

        # Ajouter un hook temporaire
        hook = model.features[layer_idx].register_forward_hook(zero_out_feature)

        # Faire une prédiction avec la feature désactivée
        with torch.no_grad():
            output_disabled = model(input_tensor)
            probs_disabled = torch.nn.functional.softmax(output_disabled, dim=1)
            new_score = probs_disabled[0, pred_class].item()

        # Supprimer le hook
        hook.remove()

        # Calcul de l'importance
        importance = original_score - new_score
        feature_importance[feature_idx] = importance

        # Affichage de progression
        print(f"Feature {feature_idx+1}/{num_features} - Importance: {importance:.4f}")

    # Trier les features par importance décroissante
    sorted_importance = dict(sorted(feature_importance.items(), key=lambda item: item[1], reverse=True))

    return sorted_importance


In [ ]:
def processPicture(global_dictionary : dict, picture_path : str = "/content/drive/My Drive/PER/data", output_pictures_path :str = "/content/drive/My Drive/PER/save"):
    local_dictionary = {}
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    image = Image.open(picture_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)
    output = model(input_tensor)
    pred_class = torch.argmax(output, dim=1).item()
    probs = torch.nn.functional.softmax(output, dim=1)

    local_dictionary[CLASSE_PREDICTED] = pred_class
    local_dictionary[PROBABILITY] = probs[0, pred_class]

    composite = EpsilonPlusFlat([SequentialMergeBatchNorm()])
    attribution = CondAttribution(model, no_param_grad=True)

    features_dict = {}
    layers_heatmaps = {}
    for layer_idx, num_features in features_per_layer.items():
        all_heatmaps = []
        num_feature_per_batch = 8
        index = 0
        borne_sup = 0
        while borne_sup != num_features:
            borne_sup = min((index+1)*num_feature_per_batch, num_features)
            conditions = [{"y": [40], "features.40": [j]} for j in range(index*num_feature_per_batch, borne_sup)]
            heatmaps, _, _, _ = attribution(input_tensor, conditions, composite)
            all_heatmaps.append(heatmaps)
            index += 1
        heatmaps = np.concatenate(all_heatmaps, axis=0)
        layers_heatmaps[layer_idx] = heatmaps

        importance_dict = compute_feature_importance(model, input_tensor, layer_idx=layer_idx, num_features=num_features, pred_class=pred_class)

        features_dict[layer_idx] = importance_dict

    # Normalisation globale sur toutes les heatmaps
    min_value = min([heatmaps.min() for heatmaps in layers_heatmaps.values()])
    max_value = max([heatmaps.max() for heatmaps in layers_heatmaps.values()])
    max_value = max(abs(min_value), abs(max_value))
    min_value = -max_value

    for idx_layers, heatmaps in layers_heatmaps.items():
        save_path_folder = os.path.join(output_pictures_path, image_name)
        save_path_folder = os.path.join(save_path, str(idx_layers))
        os.makedirs(save_path_folder, exist_ok=True)
        for idx, heatmap in enumerate(heatmaps):
            fig, ax = plt.subplots(figsize=(4, 4))
            cax = ax.imshow(heatmap, cmap="seismic", vmin=min_value, vmax=max_value)

            cbar = fig.colorbar(cax, ax=ax, fraction=0.046, pad=0.04)
            cbar.set_label("Activation Score")
            save_path = os.path.join(save_path_folder, f"{idx_layers}_{idx+1}.jpeg")
            plt.savefig(save_path, bbox_inches='tight')
            plt.close(fig)

    local_dictionary[FEATURES] = features_dict
    global_dictionary[image_name] = local_dictionary
    return global_dictionary

In [ ]:
def create_json_from_data(data, output_filename):
    """
    Fonction qui prend un dictionnaire de données avec des informations sur des images,
    et les sauvegarde dans un fichier JSON structuré.

    :param data: Dictionnaire avec les données des images.
    :param output_filename: Nom du fichier JSON à créer.
    """
    # Créer un dictionnaire pour les données au format désiré
    image_data = {}

    for image_name, info in data.items():
        # Extraire les informations : classe, probabilité, et dictionnaire de features
        classe = info['classe']
        probability = info['probability']
        features = info['features']

        # Ajouter ces informations dans le dictionnaire final
        image_data[image_name] = {
            'classe': classe,
            'probability': probability,
            'features': features
        }

    # Sauvegarder les données dans un fichier JSON
    with open(output_filename, 'w') as json_file:
        json.dump(image_data, json_file, indent=4)

    print(f"Le fichier JSON '{output_filename}' a été créé avec succès.")

In [ ]:
input_folder = "/content/drive/My Drive/PER/save"
global_dictionary = {}

image_paths = glob(os.path.join(input_folder, "*.jpeg"))

for image_path in image_paths:
    global_dictionary = processPicture(global_dictionary, image_path)
create_json_from_data(global_dictionary, "/content/drive/My Drive/PER/data.json")